# Project: Category prediction of IKEA furniture

## Data Visualization

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline
plt.style.use('ggplot') # emulate pretty r-style plots

In [150]:
#read the data and delete the index of entry
IKEA_df = pd.read_csv('IKEA_SA_Furniture_Web_Scrapings_sss.csv')
IKEA_df = IKEA_df.drop(IKEA_df.columns[0],axis =1)

In [4]:
IKEA_df.shape

(3694, 13)

We have 3694 samples and 12 fearues together with target catrgory as follows:

In [5]:
# data type  
IKEA_df.dtypes

item_id                int64
name                  object
category              object
price                float64
old_price             object
sellable_online         bool
link                  object
other_colors          object
short_description     object
designer              object
depth                float64
height               float64
width                float64
dtype: object

In [6]:
IKEA_df.count()

item_id              3694
name                 3694
category             3694
price                3694
old_price            3694
sellable_online      3694
link                 3694
other_colors         3694
short_description    3694
designer             3694
depth                2231
height               2706
width                3105
dtype: int64

We can see some data is missing in feature depth, height and width 

In [7]:
# maybe make this into a pretty pie/bar chart
IKEA_df.describe().round(2)

,item_id,price,depth,height,width
count,3694.00,3694.00,2231.00,2706.00,3105.00
mean,48632396.79,1078.21,54.38,101.68,104.47
std,28887094.10,1374.65,29.96,61.10,71.13
min,58487.00,3.00,1.00,1.00,1.00
25%,20390574.00,180.90,38.00,67.00,60.00
50%,49288078.00,544.70,47.00,83.00,80.00
75%,70403572.75,1429.50,60.00,124.00,140.00
max,99932615.00,9585.00,257.00,700.00,420.00


In [8]:
nun =  IKEA_df['category'].nunique()
print(IKEA_df['category'].unique())
print('There are ' + str(nun) + ' categories in total' )

['Bar furniture' 'Beds' 'Bookcases & shelving units'
 'Cabinets & cupboards' 'Café furniture' 'Chairs'
 'Chests of drawers & drawer units' "Children's furniture"
 'Nursery furniture' 'Outdoor furniture' 'Room dividers'
 'Sideboards, buffets & console tables' 'Sofas & armchairs'
 'Tables & desks' 'Trolleys' 'TV & media furniture' 'Wardrobes']
There are 17 categories in total


In [9]:
nun =  IKEA_df['designer'].nunique()
#print(IKEA_df['designer'].unique())
print('There are ' + str(nun) + ' designers in total' )

There are 381 designers in total


In [170]:
IKEA_df.groupby(['designer','category'])['item_id'].count().reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(10)

,designer,category,count
374,IKEA of Sweden,Bookcases & shelving units,256
375,IKEA of Sweden,Cabinets & cupboards,96
386,IKEA of Sweden,Tables & desks,89
385,IKEA of Sweden,TV & media furniture,78
259,Ehlén Johansson,Sofas & armchairs,74
377,IKEA of Sweden,Chairs,68
427,IKEA of Sweden/Ehlén Johansson,Wardrobes,62
273,Ehlén Johansson/IKEA of Sweden,Wardrobes,61
307,Francis Cayouette,Sofas & armchairs,53
753,Ola Wihlborg,Sofas & armchairs,48


In [171]:
IKEA_df['designer_len'] = IKEA_df['designer'].str.len() 
IKEA_df['designer_len'].describe()

count    3694.000000
mean     33.755820  
std      86.016629  
min      3.000000   
25%      14.000000  
50%      15.000000  
75%      28.000000  
max      1261.000000
Name: designer_len, dtype: float64

In [11]:
IKEA_df[IKEA_df['designer_len'] > 50].shape

(166, 14)

In [12]:
IKEA_df[IKEA_df['designer_len'] > 50].head(2)

,item_id,name,category,price,old_price,sellable_online,link,other_colors,short_description,designer,depth,height,width,designer_len
51,50468953,VATTVIKEN,Beds,995.0,No old price,True,https://www.ikea.com/sa/en/p/vattviken-armchai...,No,Armchair-bed,504.689.53 Small and easy-to-place chair-bed w...,83.0,86.0,92.0,403
77,90331091,RIDABU,Beds,100.0,SR 125,True,https://www.ikea.com/sa/en/p/ridabu-mirror-doo...,No,"Mirror door, 40x120 cm",903.310.91 The door can be hung with the openi...,NaN,120.0,40.0,199


We can see some data in desinger is actually long description of the furniture, we can replace them with "designer unknowm", but  this might cause to misleading that those furniture are designed by the same person, so considering there are not much of them, we keep the origin ones as designer. 

In [13]:
# correlation / scatter matrix here

## Data Cleaning

In [151]:
#convert object to boolean
IKEA_df['other_colors'] = IKEA_df['other_colors'].map({'Yes': True, 'No': False})

With following observations, we can conclude that all NaN in dataset concerning the data size

In [152]:
IKEA_df[IKEA_df.isnull().any(axis=1)].shape

(1795, 13)

In [153]:
size = ['depth','height','width']
IKEA_df[IKEA_df[size].isnull().any(axis=1)].shape

(1795, 13)

In [160]:
IKEA_df[IKEA_df[IKEA_df.columns[~IKEA_df.columns.isin(size)]].isnull().any(axis=1)].shape

(0, 13)

In [162]:
IKEA_df[IKEA_df[size].isnull().all(axis=1)].head(40)

,item_id,name,category,price,old_price,sellable_online,link,other_colors,short_description,designer,depth,height,width
2,9333523,NORDVIKEN / NORDVIKEN,Bar furniture,2095.0,No old price,False,https://www.ikea.com/sa/en/p/nordviken-nordviken-bar-table-and-4-bar-stools-black-black-s09333523/,False,Bar table and 4 bar stools,Francis Cayouette,NaN,NaN,NaN
9,69304221,EKEDALEN / EKEDALEN,Bar furniture,2176.0,"SR 2,375",True,https://www.ikea.com/sa/en/p/ekedalen-ekedalen-bar-table-and-4-bar-stools-dark-brown-orrsta-light-grey-s69304221/,False,Bar table and 4 bar stools,Ehlén Johansson,NaN,NaN,NaN
28,29297227,STENSELE / NORRARYD,Bar furniture,1340.0,No old price,True,https://www.ikea.com/sa/en/p/stensele-norraryd-bar-table-and-2-bar-stools-anthracite-anthracite-black-s29297227/,False,Bar table and 2 bar stools,Nike Karlsson/Maja Ganszyniec,NaN,NaN,NaN
32,19297275,NORRÅKER / NORRÅKER,Bar furniture,1266.0,"SR 1,385",True,https://www.ikea.com/sa/en/p/norraker-norraker-bar-table-and-2-bar-stools-birch-birch-s19297275/,False,"Bar table and 2 bar stools, 74 cm",J Karlsson/N Karlsson,NaN,NaN,NaN
34,20336619,HENRIKSDAL,Bar furniture,40.0,No old price,True,https://www.ikea.com/sa/en/p/henriksdal-cover-for-bar-stool-with-backrest-graesbo-white-20336619/,False,Cover for bar stool with backrest,IKEA of Sweden,NaN,NaN,NaN
38,29304223,EKEDALEN / EKEDALEN,Bar furniture,2375.0,No old price,True,https://www.ikea.com/sa/en/p/ekedalen-ekedalen-bar-table-and-4-bar-stools-white-orrsta-light-grey-s29304223/,False,"Bar table and 4 bar stools, 120 cm",Ehlén Johansson,NaN,NaN,NaN
39,49304217,EKEDALEN / EKEDALEN,Bar furniture,2176.0,"SR 2,375",True,https://www.ikea.com/sa/en/p/ekedalen-ekedalen-bar-table-and-4-bar-stools-brown-orrsta-light-grey-s49304217/,False,"Bar table and 4 bar stools, 120 cm",Ehlén Johansson,NaN,NaN,NaN
42,89297272,NORRÅKER / NORRARYD,Bar furniture,1266.0,"SR 1,385",True,https://www.ikea.com/sa/en/p/norraker-norraryd-bar-table-and-2-bar-stools-birch-black-s89297272/,False,"Bar table and 2 bar stools, 74 cm",Nike Karlsson/J Karlsson/N Karlsson,NaN,NaN,NaN
45,49297274,NORRÅKER / NORRÅKER,Bar furniture,1266.0,"SR 1,385",True,https://www.ikea.com/sa/en/p/norraker-norraker-bar-table-and-2-bar-stools-birch-black-s49297274/,False,"Bar table and 2 bar stools, 74 cm",J Karlsson/N Karlsson,NaN,NaN,NaN
104,44361010,BRYNE,Beds,49.0,No old price,True,https://www.ikea.com/sa/en/p/bryne-net-white-44361010/,False,Net,443.610.10 Easy to keep clean since you can remove the fabric and wash it by machine.NEVER use for infants and young children.WARNING! Entanglement and strangulation hazard.The net is not intended to be used as protection against mosquitos or other insects.,NaN,NaN,NaN


### NaN in size data

According to our observation to the datasAccording to our observation to the dataset, compared with the product size shown in the website, we have the following conclusions. There are several main reasons why size data is missing \.
 First apparently some data are not successfully scraped from the website and neither be recorded in the short description, it is impossible to recover those. \
Second, some items are a set of different furniture, so there is no product size. \
Third, some items have variable size, for example, its width can be 190 to 220 cm. Although those data are recorded in short description, its hard to extract the exact size for all those items. The biggest problem is that we can’t tell to which of three, depth, height and width , the number belongs, since the it is often that data is incomplete in short description.\
The last, there are some small items which measured in mm instead of cm and instead of height, the diameter is measured and recorded in short description.


In [ ]:
#split description and size data in feature short description
IKEA_df['short_description'].str.rsplit(',', n=1,expand = True)
IKEA_df['size']= IKEA_df['short_description'].str.rsplit(',', n=1).str[1]
IKEA_df['short_description']= IKEA_df['short_description'].str.rsplit(',', n=1).str[0]

In [ ]:
#split numbers and unit
IKEA_df['unit'] = IKEA_df['size'].str.replace('\d+|-|x', ' ')
IKEA_df['size_nounit'] = IKEA_df['size'].str.replace('cm|mm', '')

Following result shows as we find that indeed all small items don’t have height and considering it is convenient to extract the single numbers in short description representing the diameter, we fill the NaN in height of small items with data recorded in short description. Furthermore, we find that size data of small items, of which the diameter is measured with mm, is actually in mm according to the link, therefore we correct those.  

In [ ]:
IKEA_df[IKEA_df['unit'].str.contains('mm') & IKEA_df[size].isnull().any(axis=1)].shape

In [ ]:
IKEA_df[IKEA_df['unit'].str.contains('mm') & IKEA_df['height'].isnull()].shape

In [ ]:
IKEA_df['height'] = np.where(IKEA_df['unit'].str.contains('mm') , IKEA_df.size_nounit, IKEA_df.height)
IKEA_df['height'] = np.where(IKEA_df['unit'].str.contains('mm'), IKEA_df.height.astype(float)/10, IKEA_df.height)
IKEA_df['depth'] = np.where(IKEA_df['unit'].str.contains('mm'), IKEA_df.depth.astype(float)/10, IKEA_df.depth)
IKEA_df['width'] = np.where(IKEA_df['unit'].str.contains('mm'), IKEA_df.width.astype(float)/10, IKEA_df.depth)